## Project Goal/Question: How might we use self-reported global emissions data to predict Scope 3 (full supply chain) emissions values for similar companies missing emissions data?

Note: This practicum project was a partnership with a private company, so I have omitted outputs for confidentiality reasons.
We started with exploratory data analysis (EDA) to get a better sense of the data and how best to proceed.

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight')
import os

In [ ]:
#loading dataset
df = pd.read_csv('C6_5.csv')
df.columns = ['Account', 'Organization', 'Country',	'Public',	'Response_received_date', 'Primary_activity', 'Primary_sector',	'Primary_industry', 'Primary_questionnaire_sector', 'Row', 'RowName', 'C6_5_Evaluation_status', 'C6_5_Metric_tonnes_CO2', 'C6.5_Emissions_calculation_methodology', 'C6.5_Percentage_missions_calculated',	'C6.5_Exclusions']
df.info()

In [ ]:
#checking missing values
df.isnull().sum()

In [ ]:
# Convert Metric tonnes CO2 column to numeric
df['C6_5_Metric_tonnes_CO2'] = pd.to_numeric(df['C6_5_Metric_tonnes_CO2'], errors = 'coerce')

In [ ]:
#replace missing values for Evaluation status with "Not Indicated"
df['C6_5_Evaluation_status'].fillna("Not Indicated", inplace=True)
df.isnull().sum()

In [ ]:
# Group by accounts to see how many accounts are in each Eval status
df_sector = df.groupby(['Primary_sector']).count().reset_index()
df_sector

In [ ]:
#creating a dataframe grouping by sector and eval status
df_status = df.groupby(['Primary_sector','C6_5_Evaluation_status'])['Account'].count().reset_index()
df_status

In [ ]:
df_status['SectorTotal'] = df_status['Account'].groupby(df_status['Primary_sector']).transform('sum')
df_status

In [ ]:
df_status['Percent'] = df_status['Account']/df_status['SectorTotal']*100
df_status

In [ ]:
#checking missing values for Percent colum
df_status.isnull().sum()

In [ ]:
#visualizing percentages by sector
sns.set_theme(style="whitegrid")
f, ax = plt.subplots(figsize=(6, 20))
reporting = df_status.sort_values("Percent", ascending=False)
viz = sns.barplot(x="Percent", y="Primary_sector", data=reporting, hue = "C6_5_Evaluation_status", palette="mako", edgecolor=".1", dodge=False)
sns.move_legend(viz, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
#adjusting order of Evaluation Status categories
sns.set_theme(style="whitegrid")
f, ax = plt.subplots(figsize=(6, 20))
viz2 = sns.barplot(x="Percent", y="Primary_sector", data=reporting, hue_order = ["Question not applicable", "Not Indicated", "Not evaluated", "Not relevant, explanation provided", "Relevant, not yet calculated", "Not relevant, calculated", "Relevant, calculated"], hue = "C6_5_Evaluation_status", palette="mako", edgecolor=".1", dodge=False)
sns.move_legend(viz2, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
#checking back to actual total numbers of orgs in each sector/status
df.groupby('Primary_sector').size()

In [ ]:
#checking number of orgs providing calculations
calculated = ["Not relevant, calculated", "Relevant, calculated"]
df_co2 = df[df['C6_5_Evaluation_status'].isin(calculated)]
df_co2.info()

In [ ]:
df_status2 = df.groupby(['C6_5_Evaluation_status'])['Account'].count().reset_index()
df_status2

In [ ]:
#looking at average CO2 for orgs reporting
df_mean = df_co2.groupby(['C6_5_Evaluation_status'])['C6_5_Metric_tonnes_CO2'].mean().reset_index()
df_mean

###Summary of findings so far

Logging and rubber tapping is the only sector with more than 50% of the companies reporting/calcuating CO2 emissions, although there are also a relatively small number of organizations in that category (34). Tobacco is close and then we have a pretty big dropoff to < 40% of organizations in each category reporting calculations of CO2 emissions. Based on these relatively low percentages I wonder how helpful Primary sector will be as a predictor, so perhaps some of the RowName/Greenhouse Gas Protocol categories will be more informative in terms of predicting emissions values. Government agencies are also a huge outlier with no evaluation status even reported, though there are also only 17 government agencies in the dataset.

We decided to focus in on the companies that had indicated Scope 3 emissions were relevant and calculated, merging in additional details about their verification processes next.


In [ ]:
relevant_calc = df[df['C6_5_Evaluation_status']=='Relevant, calculated']
relevant_calc.info()

In [ ]:
#checking missing values
relevant_calc.isna().sum()

In [ ]:
#getting number of unique accounts
df['Account'].nunique()

In [ ]:
df_10_1 = pd.read_csv('C10.1c.csv')
df_10_1.info()

In [ ]:
df_10_1.columns = [
    'Account', 'Organization', 'Country',	'Public',	
    'Response_received_date', 'Primary_activity', 'Primary_sector',	
    'Primary_industry', 'Primary_questionnaire_sector', 'C10_1c_Row', 'C10_1c_RowName', 
    'C10_1c_Scope_3_category', 'C10_1c_Verification_cycle', 
    'C10_1c_Verification_status', 'C10_1c_Verification_type', 'C10_1c_Statement',
    'C10_1c_Reference', 'C10_1c_standard', 'C10_1c_Proportion_of_emissions_verified']
df_10_1.head()

In [ ]:
#looking at response options for verification status
df_10_1['C10_1c_Verification_status'].unique()

In [ ]:
df_10_1['C10_1c_Verification_status'].value_counts()

In [ ]:
#filtering to only organizations with Complete verification status
complete = df_10_1[df_10_1['C10_1c_Verification_status']=='Complete'].drop_duplicates(subset=['Account'])
complete.head()

In [ ]:
#Breakdown of Complete companies by verification type
complete['C10_1c_Verification_type'].value_counts()

In [ ]:
#streamline 10_1_c down to just the columns needed for merge
mergers_10_1 = df_10_1[df_10_1['C10_1c_Verification_status']=='Complete']
mergers_10_1.drop(['Public', 'Response_received_date','C10_1c_Statement','C10_1c_Reference', 'C10_1c_standard'], axis=1, inplace=True)
mergers_10_1.head()

In [ ]:
#comparing above numbers to total number of organizations
df.nunique()

These numbers indicate we would be able to use about 10% of the total accounts for predictions if we focused on reasonable assurance and up. It might be worthwhile to compare results of a model when the Limited Assurance category is included vs. excluded.

In [ ]:
#examining proportion of emissions verified
complete.info()

In [ ]:
complete['C10_1c_Proportion_of_emissions_verified']=pd.to_numeric(complete['C10_1c_Proportion_of_emissions_verified'])
complete['C10_1c_Proportion_of_emissions_verified'].value_counts()

Out of the 1457 accounts with Complete verification status, 86% of them indicate 100% of their emissions have been verified. If that's the case, I wonder how meaningful is verification type/level of assurance? I'm thinking this might be a reasonable argument for using those 1249 accounts for further examination. 

In [ ]:
#pivot the 6_5 dataframe to get to focus on emissions by Account
pivot_df = relevant_calc.pivot(index='Account', columns='RowName', values = ['C6_5_Metric_tonnes_CO2'])
pivot_df.head(10)
#and merge with the 10_1 verification on Account number; drop unneeded columns and start looking at calculation status, etc. by sector for the most "reliable" sectors

In [ ]:
#fill in NaN values with zeros
pivot_df.fillna(0, inplace=True)
pivot_df.head()

In [ ]:
#Summing emissions values
total_scope_3 = pivot_df

total_scope_3['Total Scope 3 Emissions'] = total_scope_3.sum(axis=1)

total_scope_3.head()

In [ ]:
#streamline 10_1_c down to just the columns needed for merge
mergers_10_1 = df_10_1[df_10_1['C10_1c_Verification_status']=='Complete'].copy()
mergers_10_1.drop(['Public', 'Response_received_date','C10_1c_Statement','C10_1c_Reference', 'C10_1c_standard'], axis=1, inplace=True)
mergers_10_1.head()

In [ ]:
combined = pd.merge(mergers_10_1, total_scope_3 , on = 'Account', how = 'left')
combined.head()

In [ ]:
combined.info()

In [ ]:
combined['C10_1c_Verification_type'].value_counts()

In [ ]:
#checking the handful of missing values in the Verification type category; the spreadsheet appears to have values for these, so not sure if it was an issue with the pivot. Still need to figure out how best to troubleshoot that part
missing_verify = combined[combined['C10_1c_Verification_type'].isna()]
pd.set_option('display.max_columns', 50)
missing_verify.head(40)

In [ ]:
df_sectors = combined
df_sectors.head()

In [ ]:
#dropping some unneeded columns.
df_sectors.drop(['Primary_questionnaire_sector', 'C10_1c_Row', 'C10_1c_RowName', 'C10_1c_Scope_3_category', 'C10_1c_Verification_cycle', 'C10_1c_Verification_status'], axis=1, inplace=True)
df_sectors.head()

In [ ]:
#renaming all columns to try to remove the parentheses issue post-merge
df_sectors.columns = ['Account', 'Organization', 'Country', 'Primary_activity', 'Primary_sector', 'Primary_industry', 'C10_1c_Verification_type', 'C10_1c_Proportion_of_emissions_verified', 
                     'C6_5_Metric_tonnes_CO2, Business travel', 'C6_5_Metric_tonnes_CO2, Capital goods', 'C6_5_Metric_tonnes_CO2, Downstream leased assets', 'C6_5_Metric_tonnes_CO2, Downstream transportation and distribution',
                     'C6_5_Metric_tonnes_CO2, Employee commuting', 'C6_5_Metric_tonnes_CO2, End of life treatment of sold products', 'C6_5_Metric_tonnes_CO2, Franchises', 'C6_5_Metric_tonnes_CO2, Fuel-and-energy-related activities_not included in Scope 1 or 2',
                     'C6_5_Metric_tonnes_CO2, Investments', 'C6_5_Metric_tonnes_CO2, Other_downstream', 'C6_5_Metric_tonnes_CO2, Other_upstream', 'C6_5_Metric_tonnes_CO2, Processing of sold products', 'C6_5_Metric_tonnes_CO2, Purchased goods and services', 'C6_5_Metric_tonnes_CO2, Upstream leased assets',
                     'C6_5_Metric_tonnes_CO2, Upstream transportation and distribution', 'C6_5_Metric_tonnes_CO2, Use of sold products', 'C6_5_Metric_tonnes_CO2, Waste generated in operations', 'Total_Scope3_Emissions']
df_sectors['Primary_sector'].value_counts()

In [ ]:
#testing for differences by verification status, starting with financial services since it has the most orgs
financial_svcs = df_sectors[df_sectors['Primary_sector'] == 'Financial services']
financial_svcs.head()

In [ ]:
financial_svcs['C10_1c_Verification_type'].value_counts()

In [ ]:
missing_fs = financial_svcs[financial_svcs['C10_1c_Verification_type'].isnull()]
missing_fs

In [ ]:
#filling in the few missing values with "Unknown" for verification status
financial_svcs['C10_1c_Verification_type'].fillna('Unknown', inplace=True)
financial_svcs.info()

In [ ]:
#checking emissions data distribution for normality to determine parametric (ANOVA) vs non-parametric test to compare differences
fig, ax = plt.subplots(figsize=(15, 5))
sns.barplot(x='C10_1c_Verification_type', y='Total_Scope3_Emissions', data=financial_svcs, ax=ax)
plt.show()

In [ ]:
import scipy.stats as stats
res = stats.probplot(financial_svcs['Total_Scope3_Emissions'], plot=plt)

In [ ]:
#running Kruskal-Wallis test to check for significant differences by verification status
unknown = financial_svcs['C10_1c_Verification_type'] == 'Unknown'
limited = financial_svcs['C10_1c_Verification_type'] == 'Limited assurance'
reasonable = financial_svcs['C10_1c_Verification_type'] == 'Reasonable assurance'
moderate = financial_svcs['C10_1c_Verification_type'] == 'Moderate assurance'
high = financial_svcs['C10_1c_Verification_type'] == 'High assurance'
underway = financial_svcs['C10_1c_Verification_type'] == 'Third party verification/ assurance underway'
stats.kruskal(unknown, limited, reasonable, moderate, high, underway)

Kruskal-Wallis test was significant, indicating emissions data within Financial Services differs significantly by verification status. Based on these results we'll want to focus on either establishing some scoring criteria or limiting the data we use for prediction to the top 1-2 levels of assurance (or a bit of both).

Based on further discussion with the project team and company contact, I shifted focus to work on identifying the sectors with high “Purchased goods and services” (Category 1) and “Use of sold products” (Category 11) emissions. I set out to identify three main items:

1. Sectors with the highest Category 1 and 11 emissions
2. Sectors with the most companies reporting Category 1 and 11 emissions
3. Sectors with the highest percentage of companies reporting Category 1 and 11 emissions


In [ ]:
#filtering to focus on Purchased goods and services and Use of sold products columns
focus_cats = relevant_calc.loc[relevant_calc['RowName'].isin(['Purchased goods and services', 'Use of sold products'])]
focus_cats.head()

In [ ]:
#dropping some excess columns
focus_cats = focus_cats[['Account', 'Organization', 'Country', 'Primary_sector', 'RowName', 'C6_5_Evaluation_status', 'C6_5_Metric_tonnes_CO2', 'C6.5_Percentage_missions_calculated' ]]
focus_cats.head()

In [ ]:
#checking/addressing missing values
focus_cats.isna().sum()

In [ ]:
focus_cats.fillna('Unknown', inplace=True)

In [ ]:
#creating a pivot table grouping CO2 values by sector and category
focus_pivot = pd.pivot_table(focus_cats, values='C6_5_Metric_tonnes_CO2', index=['Primary_sector'], columns = ['RowName'], aggfunc=np.sum)
focus_pivot.head()

In [ ]:
#adding a column to sum the values
focus_pivot['Total CO2e'] = focus_pivot.sum(axis=1)
focus_pivot.head()

In [ ]:
#suppressing scientific notation
pd.options.display.float_format = '{:.5f}'.format
focus_pivot.sort_values(by='Total CO2e', ascending=False)

Sectors with the highest CO2 emissions reported in our focus categories:

1) Electrical & electronic equipment 
2) Oil and gas processing 
3) Transportation equipment 
4) Powered machinery 
5) Chemicals

In [ ]:
#Sectors with the most companies reporting Category 1 and 11 emissions
focus_orgs = relevant_calc.loc[relevant_calc['RowName'].isin(['Purchased goods and services', 'Use of sold products'])]
focus_orgs.head()

In [ ]:
focus_orgs.groupby(by='Primary_sector').nunique().sort_values(by='Organization', ascending=False)

Sectors with the largest number of organizations reporting in key categories are:
1) Electrical & electronic equipment
2) Financial services
3) Chemicals
4) Food & beverage processing
5) Powered Machinery

In [ ]:
#getting total list of orgs
total_orgs = relevant_calc.drop_duplicates(subset='Organization')
total_orgs.info()

In [ ]:
#adding the rows with the focus categories into the total_orgs dataframe
total_orgs = pd.concat([total_orgs, focus_orgs], ignore_index=True)
total_orgs.tail()

In [ ]:
#adding column to flag if org reported emissions in the focus categories
total_orgs['FocusCategory'] = np.where(total_orgs['RowName'].isin(['Purchased goods and services', 'Use of sold products']), True, False)
total_orgs.tail()

In [ ]:
#removing duplicates so we can calculate percentage accurately
total_orgs.drop_duplicates('Organization', keep='last')
total_orgs.head()

In [ ]:
total_orgs.nunique()

In [ ]:
#calculating percentage reporting focus category emissions by sector
percentages = total_orgs.pivot_table(index='Primary_sector', values = ['FocusCategory', 'RowName'], aggfunc={'FocusCategory': 'sum', 'RowName': 'count'})
percentages.head()

In [ ]:
percentages['PercentofTotal'] = percentages['FocusCategory']/percentages['RowName']
percentages.sort_values('PercentofTotal', ascending=False)

In [ ]:
percentages['PercentofTotal'].mean()

In [ ]:
percentages['PercentofTotal'].median()

Top 5 are very different when we look at percent of total organizations reporting emissions in the focus categories:

1) Logging & rubber tapping (2 of 2 orgs) 
2) Crop farming (8 of 8 orgs) 
3) Rail transport (17 of 18 orgs) 
4) Tobacco (13 of 14 orgs) 
5) Fish & animal farming (10 of 11 orgs)

But the total number of orgs reporting emissions in those first 5 categories are all relatively small. If we look at the sectors with the largest number of orgs reporting in focus categories, percent of total orgs report are as follows:

1) Electrical & electronic equipment (66%) 
2) Financial services (66%) 
3) Chemicals (82%) 
4) Food & beverage processing (77%) 
5) Powered Machinery (66%)

All are at or above the median value for % reporting. We could potentially focus on a few of the largest sectors, and/or consider any sectors above that median % reporting.


In [ ]:
#writing to csv files
percentages.to_csv('percentages.csv')
total_orgs.to_csv('total_orgs.csv')
focus_orgs.to_csv('focus_orgs.csv')

In [ ]:
# Generate column identifying organizations that reported emissions in both focus categories
x=0
result = []
for value in relevant_calc['RowName']:
    if value in(['Purchased goods and services']):
      result.append(x+1)
    elif value in (['Use of sold products']):
      result.append(x+1)
    else:
        result.append(x)
      
relevant_calc['FocusCats'] = result
relevant_calc.tail()

In [ ]:
focus_cats = relevant_calc.loc[relevant_calc['FocusCats'] >= 1]
focus_cats.info()

In [ ]:
both_cats = focus_cats[focus_cats.duplicated('Organization')]
both_cats

In [ ]:
both_cats['FocusCats'] = 2
both_cats.info()

In [ ]:
focus_final = pd.concat([focus_cats, both_cats])
focus_final.tail()

In [ ]:
focus_final.drop_duplicates('Organization', keep='last', inplace=True)
focus_final.info()

In [ ]:
focus_final.drop(['Row', 'RowName', 'C6_5_Metric_tonnes_CO2', 'C6.5_Percentage_missions_calculated', 'C6.5_Exclusions'], axis=1, inplace=True)
focus_final.info()

In [ ]:
#writing list to csv files
focus_final.to_csv('focus_final.csv')

At this point in the project, we received an additional data source reporting annual revenue to incorporate into our analysis and modeling efforts. One of my teammates focused on merging in the financial data. Because the revenue data came from a completely separate source, it took considerable work to match up revenue data with the companies in our dataset. Because we opted to only use companies with revenue info in our modeling attempts, our dataset got considerably smaller (810 companies compared to nearly 3,000 in my previous "focus orgs" dataset). Another teammate had developed a "trust score" methodology based on the different verification criteria available to us in the dataset, so we incorporated it into our dataset as a potential model feature.

I started with some visualization and examining outliers in our updated dataset to help determine a modeling plan.

In [ ]:
df = pd.read_csv('full_810_frame_with_trust_score.csv')
df.info()

In [ ]:
df['primary_sector'].unique()

In [ ]:
focus_sectors = df[df['primary_sector'].isin(['Electrical & electronic equipment', 'Financial services', 'Chemicals','Food & beverage processing','Powered machinery'])]
sns.set_style("ticks")
f = plt.figure(figsize=(15, 8))
sns.boxplot(data=focus_sectors, x='primary_sector', y='cat_1')

In [ ]:
# IQR cat_1
Q1 = np.percentile(focus_sectors['cat_1'], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(focus_sectors['cat_1'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
Q3

In [ ]:
outliers = focus_sectors[focus_sectors['cat_1']> Q3]
outliers['country'].value_counts()

In [ ]:
outliers['primary_activity'].value_counts()

In [ ]:
outliers['primary_industry'].value_counts()

In [ ]:
outliers['verification_type'].value_counts()

In [ ]:
trust_check = outliers[outliers['Percent S3 verified FILTERED']>0]
trust_check

In [ ]:
f = plt.figure(figsize=(15, 8))
sns.boxplot(data=focus_sectors, x='primary_sector', y='cat_11')

In [ ]:
# IQR cat_11
Q1_11 = np.percentile(focus_sectors['cat_11'], 25,
                   interpolation = 'midpoint')
 
Q3_11 = np.percentile(focus_sectors['cat_11'], 75,
                   interpolation = 'midpoint')
IQR = Q3_11 - Q1_11
Q3_11

In [ ]:
outliers_11 = focus_sectors[focus_sectors['cat_11']> Q3]
outliers_11['verification_type'].value_counts()

In [ ]:
#dropping columns I definitely won't want in the model. Keeping one Organization column for now in case it helps to examine outliers, etc. if needed
df_trimmed = df.copy()
df_trimmed.drop(['Unnamed: 0', 'account_number', 'organization', 'statement', 'Account'], axis=1, inplace=True)

In [ ]:
#converting selected object columns to category codes
df_quant = df_trimmed.copy()
df_quant.head()

In [ ]:
for col in ['country', 'primary_activity', 'primary_sector', 'primary_industry', 'verification_type']:
  df_quant[col] = df_quant[col].astype('category')
df_quant.info()

In [ ]:
for col in ['country', 'primary_activity', 'primary_sector', 'primary_industry', 'verification_type']:
  df_quant[col] = df_quant[col].cat.codes
df_quant.head()

In [ ]:
#dropping remaining object and total emissions columns
df_quant.drop(['verification_cycle_in_place', 'report_status', 'Organization', 'total_emissions'], axis=1, inplace=True)

In [ ]:
#importing modeling libraries and scaling the features
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
scaler = StandardScaler()
fs_reg = scaler.fit_transform(df_quant) 

In [ ]:
scaled = pd.DataFrame(fs_reg, columns=['country', 'primary_activity', 'primary_sector', 'primary_industry',
       'cat_6', 'cat_2', 'cat_13', 'cat_9', 'cat_7', 'cat_12', 'cat_14',
       'cat_3', 'cat_15', 'cat_17', 'cat_16', 'cat_10', 'cat_1', 'cat_8',
       'cat_4', 'cat_11', 'cat_5', 'revenue', 'verification_type',
       'revenue / emissions', 'Percent S3 verified',
       'Percent S3 verified FILTERED'])

In [ ]:
scaled.describe()

In [ ]:
#looking at correlations with category 1 and category 11
correlation_matrix_1 = scaled.corr()
correlation_matrix_1['cat_1'].sort_values(ascending=False)

In [ ]:
correlation_matrix_11 = scaled.corr()
correlation_matrix_11['cat_11'].sort_values(ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
_ = sns.heatmap(scaled.corr(), ax=ax, annot=True, fmt='.2f')

Because we are trying to predict a continuous variable (CO2 emissions), I decided to start with multiple linear regression.

In [ ]:
#running regression model starting with predicting category 1 emissions. I'm not using other categories as features initially, but there are a couple that are highly correlated (cat_4, cat_5, and cat_7)
y = scaled['cat_1']
x = scaled[['primary_activity', 'primary_sector', 'primary_industry', 'country', 'revenue', 'Percent S3 verified FILTERED']]

In [ ]:
# Split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))
#MAE
print("Mean absolute error: %.2f" % mean_absolute_error(y_test, y_pred))

In [ ]:
#plotting residuals
from yellowbrick.regressor import ResidualsPlot
visualizer = ResidualsPlot(regr)

visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show()                 # Finalize and render the figure

In [ ]:
#normal residual QQ plot
import statsmodels.api as sm
fig = sm.qqplot(x, line='45')
plt.show()

In [ ]:
#plots indicate some heteroscedasticity; trying log transformation on the dependent variable (although wondering if this is not appropriate since the data is already scaled)
ylog = np.log1p(y)
fig, axs = plt.subplots(nrows=1, ncols=2)

#create histograms
axs[0].hist(y.dropna(), edgecolor='blue')
axs[1].hist(ylog, edgecolor='blue')

#add title to each histogram
axs[0].set_title('Original Data')
axs[1].set_title('Log-Transformed Data')

In [ ]:
#re-running model with log tranformation 

X_train, X_test, y_train, y_test = train_test_split(x, ylog, test_size=0.33, random_state=42)

# Create linear regression object
regr_log = linear_model.LinearRegression()

# Train the model using the training sets
regr_log.fit(X_train, y_train)

# Make predictions using the testing set
y_pred2 = regr_log.predict(X_test)

# The coefficients
print("Coefficients: \n", regr_log.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred2))
#MAE
print("Mean absolute error: %.2f" % mean_absolute_error(y_test, y_pred2))

In [ ]:
visualizer = ResidualsPlot(regr_log)

visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show()                 # Finalize and render the figure

In [ ]:
# trying Lasso Regression
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
# define model
model = Lasso(alpha=1.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

The Lasso regression performed well (Mean MAE: 0.388), but was using raw Scope 3 emissions values rather than emissions adjusted for revenue (metric tonnes per $1 million in annual revenue). We had agreed to use the revenue-adjusted metric as our outcome value going forward, so my next step was re-running the models with adjusted category 1/category 11 values as the targets.


In [ ]:
df = pd.read_csv('full_810_frame_with_trust_score_emissions_adjusted.csv')
#filter down to columns to be used for modeling
model_df = df[['primary_sector', 'Percent S3 verified', 'Percent S3 verified FILTERED', 'cat_1_adj', 'cat_2_adj', 'cat_3_adj', 'cat_4_adj',
               'cat_5_adj', 'cat_6_adj', 'cat_7_adj', 'cat_8_adj', 'cat_9_adj', 'cat_10_adj', 'cat_11_adj', 'cat_12_adj', 'cat_13_adj', 'cat_14_adj',
               'cat_15_adj', 'cat_16_adj', 'cat_17_adj']]
model_df.head()

In [ ]:
focus_sectors = model_df[model_df['primary_sector'].isin(['Electrical & electronic equipment', 'Financial services', 'Chemicals','Food & beverage processing','Powered machinery'])]
sns.set_style("ticks")
f = plt.figure(figsize=(15, 8))
sns.boxplot(data=focus_sectors, x='primary_sector', y='cat_1_adj')

In [ ]:
# IQR cat_1 - getting these values for possible later use
Q1_1 = np.percentile(focus_sectors['cat_1_adj'], 25,
                   interpolation = 'midpoint')
 
Q3_1 = np.percentile(focus_sectors['cat_1_adj'], 75,
                   interpolation = 'midpoint')
IQR = Q3_1 - Q1_1
Q3_1

In [ ]:
focus_sectors.reset_index()
focus_sectors.info()

In [ ]:
#seeing what our biggest oulier is for category 1 and category 11
extreme = focus_sectors[focus_sectors['cat_1_adj'] > 30000]
extreme

In [ ]:
extreme_11 = focus_sectors[focus_sectors['cat_11_adj'] > 30000]
extreme_11

In [ ]:
#trying a model with our two most extreme outliers dropped
focus_sectors_trimmed = focus_sectors[focus_sectors['cat_11_adj']< 30000]
focus_sectors_trimmed.reset_index(inplace=True)
focus_sectors_trimmed.info()

In [ ]:
f = plt.figure(figsize=(15, 8))
sns.boxplot(data=focus_sectors, x='primary_sector', y='cat_11_adj')

In [ ]:
# IQR cat_11
Q1_11 = np.percentile(focus_sectors['cat_11_adj'], 25,
                   interpolation = 'midpoint')
 
Q3_11 = np.percentile(focus_sectors['cat_11_adj'], 75,
                   interpolation = 'midpoint')
IQR = Q3_11 - Q1_11
Q3_11

In [ ]:
focus_sectors_trimmed.columns

In [ ]:
#separate into x and y dataframes for features and target
y1 = focus_sectors_trimmed['cat_1_adj']
x = focus_sectors_trimmed[['Percent S3 verified', 'Percent S3 verified FILTERED', 'cat_2_adj', 'cat_3_adj', 'cat_4_adj', 'cat_5_adj',
       'cat_6_adj', 'cat_7_adj', 'cat_8_adj', 'cat_9_adj', 'cat_10_adj',
       'cat_11_adj', 'cat_12_adj', 'cat_13_adj', 'cat_14_adj', 'cat_15_adj',
       'cat_16_adj', 'cat_17_adj']]
x.head()

In [ ]:
scaler = StandardScaler()
fs_reg = scaler.fit_transform(x) 

In [ ]:
scaled = pd.DataFrame(fs_reg, columns=['Percent S3 verified',
       'Percent S3 verified FILTERED', 'cat_2_adj', 'cat_3_adj', 'cat_4_adj', 'cat_5_adj', 'cat_6_adj', 'cat_7_adj', 'cat_8_adj', 'cat_9_adj',
       'cat_10_adj', 'cat_11_adj', 'cat_12_adj', 'cat_13_adj', 'cat_14_adj',  'cat_15_adj',  'cat_16_adj', 'cat_17_adj'])
scaled.describe()

In [ ]:
#adding back primary sector
sectors = focus_sectors_trimmed['primary_sector']
sectors.head()

In [ ]:
combined_df = scaled.join(sectors)
combined_df.head()

In [ ]:
# Creating dummy variables
df_dummies = combined_df
df_dummies = pd.get_dummies(df_dummies, columns=['primary_sector'], prefix=['sector'])
df_dummies.head()

In [ ]:
# Split the data into training/testing sets
x2 = df_dummies
X_train, X_test, y_train, y_test = train_test_split(x2, y1, test_size=0.33, random_state=42)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))
#MAE
print("Mean absolute error: %.2f" % mean_absolute_error(y_test, y_pred))

In [ ]:
#trying log scale transformation since model metrics (RMSE, MAE) are poor
xlog = np.log1p(x2)
fig, axs = plt.subplots(nrows=1, ncols=2)

#create histograms
axs[0].hist(x2.dropna(), edgecolor='blue')
axs[1].hist(xlog, edgecolor='blue')

#add title to each histogram
axs[0].set_title('Original Data')
axs[1].set_title('Log-Transformed Data')

In [ ]:
#re-running model with log tranformation 

X_train, X_test, y_train, y_test = train_test_split(xlog, y1, test_size=0.33, random_state=42)

# Create linear regression object
regr_log = linear_model.LinearRegression()

# Train the model using the training sets
regr_log.fit(X_train, y_train)

# Make predictions using the testing set
y_pred2 = regr_log.predict(X_test)

# The coefficients
print("Coefficients: \n", regr_log.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred2))
#MAE
print("Mean absolute error: %.2f" % mean_absolute_error(y_test, y_pred2))

In [ ]:
#trying SVM because MSE/MAE are still really high
from sklearn import svm
from sklearn.metrics import mean_squared_error as mse
X_SVMtrain, X_SVMtest, y_SVMtrain, y_SVMtest = train_test_split(x2, y1, test_size=0.33, random_state=42)
clf = svm.SVR(kernel="rbf", C=50, gamma="auto", degree=3, epsilon=0, coef0=0.1)
clf.fit(X_SVMtrain, y_SVMtrain)
print('R2 on test data:', + clf.score(X_SVMtest, y_SVMtest))
y_pred = clf.predict(X_SVMtest)
print('RMSE of prediction:', + mse(y_pred, y_SVMtest, squared = False))
print('mean of y_test:', + y_SVMtest.mean())
print('MAE', + mean_absolute_error(y_pred, y_SVMtest))
res = y_pred - y_SVMtest
plt.scatter(y_pred, res)
plt.show()
plt.scatter(y_pred, y_SVMtest)
plt.show()

In [ ]:
#Lasso Regression
from sklearn.linear_model import Lasso
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from numpy import absolute

In [ ]:
model = Lasso(alpha=1.0)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, x2, y1, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
#still really high MAE values. Revisiting the correlation matrix to potentially trim some features
correlation_matrix = focus_sectors_trimmed.corr()
correlation_matrix['cat_1_adj'].sort_values(ascending=False)

In [ ]:
x_trim = x2[['cat_2_adj', 'cat_15_adj', 'cat_14_adj', 'cat_4_adj', 'cat_5_adj']]
x_trim.head()

In [ ]:
#re-running lasso with trimmed feature set
model = Lasso(alpha=1.0)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, x_trim, y1, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
#MAE is still high; trying Ridge regression
from sklearn.linear_model import Ridge

In [ ]:
ridgemodel = Ridge(alpha=1.0)
# evaluate model
scores = cross_val_score(ridgemodel, x_trim, y1, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
#retrying SVM with trimmed feature set
X_SVMtrain, X_SVMtest, y_SVMtrain, y_SVMtest = train_test_split(x_trim, y1, test_size=0.33, random_state=42)
clf = svm.SVR(kernel="rbf", C=50, gamma="auto", degree=3, epsilon=0, coef0=0.1)
clf.fit(X_SVMtrain, y_SVMtrain)
print('R2 on test data:', + clf.score(X_SVMtest, y_SVMtest))
y_pred = clf.predict(X_SVMtest)
print('RMSE of prediction:', + mse(y_pred, y_SVMtest, squared = False))
print('mean of y_test:', + y_SVMtest.mean())
print('MAE', + mean_absolute_error(y_pred, y_SVMtest))
res = y_pred - y_SVMtest
plt.scatter(y_pred, res)
plt.show()
plt.scatter(y_pred, y_SVMtest)
plt.show()

Mean Absolute Error (MAE) scores continued to be pretty high on all of the regression and SVM modeling attempts, so none of these model options were looking promising. The spread of the dataset seemed to be a possible issue, given that there were a handful of organizations that had extremely high adjusted category 1 and 11 emissions values. There are enough that it didn't make sense to consider them truly outliers and adjust or throw them out, particularly considering we know there were hundreds more companies that had reported emissions data but couldn't be matched to revenue data; it's possible those gaps could get filled in with more data. I decided to explore binning our target values as a potential solution. I thought if we could bin the adjusted emissions values into reasonably meaningful ranges, it would still be helpful to our company contact and would enable me to try out some classifier models.

In [ ]:
df = pd.read_csv('full_810_frame_with_trust_score_emissions_adjusted.csv')
df.info()

In [ ]:
df['Data_Year'] = 2020
df.head()

In [ ]:
df['cat_1_adj'].describe()

In [ ]:
#adding in 2021 data
df_21 = pd.read_csv('2021_data.csv')
df_21.head()

In [ ]:
#getting all columns aligned and names consistent for merging
df.columns

In [ ]:
df.rename(columns={'revenue': 'revenue_final'})

In [ ]:
df_20 = df
df_20.drop(['cat_6',
       'cat_2', 'cat_13', 'cat_9', 'cat_7', 'cat_12', 'cat_14', 'cat_3',
       'cat_15', 'cat_17', 'cat_16', 'cat_10', 'cat_1', 'cat_8', 'cat_4',
       'cat_11', 'cat_5', 'verification_cycle_in_place',
       'report_status', 'verification_type', 'statement', 'total_emissions',
       'revenue / emissions', 'Account', 'Organization', 'Percent S3 verified',
       'Percent S3 verified FILTERED'], axis=1, inplace=True)

In [ ]:
df_20.columns

In [ ]:
df_20.rename(columns={'revenue': 'revenue_final'}, inplace=True)

In [ ]:
df_20.columns

In [ ]:
df_21.rename(columns={'country_area': 'country'}, inplace=True)

In [ ]:
df_21.columns

In [ ]:
df_both_years = pd.concat([df, df_21], ignore_index=True)
df_both_years

In [ ]:
df_both_years['cat_1_adj'].describe()

In [ ]:
df_21['cat_1_adj'].describe()

In [ ]:
#limiting to the five focus sectors we had identified as having a large number/high percentage of companies reporting category 1 and 11 emissions and dummy coding categorical variables
x_bin = df_both_years[df_both_years['primary_sector'].isin(['Electrical & electronic equipment', 'Financial services', 'Chemicals','Food & beverage processing','Powered machinery'])].copy()
x_bin = pd.get_dummies(x_bin, columns=['country', 'primary_sector'], prefix=['country_', 'sect_'])
x_bin.info()

In [ ]:
#looking at distribution of cat_1_adj to determine bin values
x_bin['cat_1_adj'].describe()

In [ ]:
x_bin['bins'] = pd.cut(x=x_bin['cat_1_adj'], bins=[0, .001, 50, 250, 1000000],
                    labels=['No Category 1', '< 50 MT per million', '<250 MT per million', '250 MT per million or more'])
x_bin.head()

In [ ]:
x_bin.bins.fillna('No Category 1', inplace=True)
x_bin.head()

In [ ]:
x_bin['bins'].value_counts()

In [ ]:
x_bin.columns

In [ ]:
#splitting into features and targets before scaling
y_class = x_bin['bins']
x_class = x_bin
x_class.drop(['Unnamed: 0', 'account_number', 'organization', 'primary_activity',
       'primary_industry', 'revenue_final', 'cat_1_adj', 'bins'], axis=1, inplace=True)

In [ ]:
cols = x_class.columns

In [ ]:
scaler = StandardScaler()
bin_scale = scaler.fit_transform(x_class) 

In [ ]:
x_class_scaled = pd.DataFrame(bin_scale, columns=cols)
x_class_scaled.describe()

In [ ]:
y_class.astype('category').cat.codes

In [ ]:
#trying an ADABoost model first because a teammate had seen a bit of promise in another modeling attempt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn import metrics

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_class_scaled, y_class, test_size=0.2)
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
abc_model = abc.fit(X_train, y_train) 
y_pred = abc_model.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

In [ ]:
#trying Random Forest classifier since ADABoost accuracy was low
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=4, random_state=42)
rfc.fit(X_train, y_train)

print(rfc.score(X_train, y_train))
print(rfc.score(X_test, y_test))

The Random Forest showed some promising accuracy scores (initially .80 for training and .75 for test). Values shifted a bit when train/test split was rerun; not surprising given this dataset is on the small side.

In [ ]:
#looking at feature importances
import sys
!pip install scikit-plot

In [ ]:
from scikitplot.estimators import plot_feature_importances
plt.rcParams["figure.figsize"] = (20, 7)
plot_feature_importances(rfc, feature_names=x_class_scaled.columns, x_tick_rotation=75)

In [ ]:
#dropping some features just to see if it helps the training scores improve at all
x_reduced = x_class[['cat_4_adj', 'cat_3_adj', 'sect__Financial services', 'cat_5_adj','cat_6_adj', 'cat_2_adj', 'cat_12_adj', 'cat_11_adj', 'cat_9_adj', 'cat_7_adj', 'cat_6_adj']]
x_reduced.head()

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(x_reduced, y_class, test_size=0.2)
rfc2 = RandomForestClassifier(max_depth=4, random_state=42)
rfc2.fit(X_train2, y_train2)

print(rfc2.score(X_train2, y_train2))
print(rfc2.score(X_test2, y_test2))

The train/test scores are a bit closer than the original model run (.79 train, .73 test).

In [ ]:
#trying the original Random Forest model for predicting Category 11 values
x_bin.columns

In [ ]:
x_bin_11 = df_both_years[df_both_years['primary_sector'].isin(['Electrical & electronic equipment', 'Financial services', 'Chemicals','Food & beverage processing','Powered machinery'])].copy()

In [ ]:
x_bin_11.info()

In [ ]:
x_bin_11 = pd.get_dummies(x_bin_11, columns=['country', 'primary_sector'], prefix=['country_', 'sect_'])

In [ ]:
x_bin_11['bins'] = pd.cut(x=x_bin_11['cat_11_adj'], bins=[0, .001, 50, 250, 1000000],
                    labels=['No Category 11', '< 50 MT per million', '<250 MT per million', '250 MT per million or more'])
x_bin_11.head()

In [ ]:
x_bin_11.bins.fillna('No Category 11', inplace=True)
x_bin_11.head()

In [ ]:
x_bin_11['bins'].value_counts()

In [ ]:
x_bin_11.columns

In [ ]:
y_class_11 = x_bin_11['bins']
x_class_11 = x_bin_11
x_class_11.drop(['Unnamed: 0', 'account_number', 'organization', 'primary_activity',
       'primary_industry', 'revenue_final', 'cat_11_adj','bins'], axis=1, inplace=True)

In [ ]:
cols_11 = x_class_11.columns

In [ ]:
scaler_11 = StandardScaler()
bin_scale_11 = scaler.fit_transform(x_class_11)

In [ ]:
x_class_scaled_11 = pd.DataFrame(bin_scale_11, columns=cols_11)
x_class_scaled_11.describe()

In [ ]:
y_class_11.astype('category').cat.codes

In [ ]:
X_train_11, X_test_11, y_train_11, y_test_11 = train_test_split(x_class_scaled_11, y_class_11, test_size=0.2)

rfc_11 = RandomForestClassifier(max_depth=4, random_state=42)
rfc_11.fit(X_train_11, y_train_11)

print(rfc_11.score(X_train_11, y_train_11))
print(rfc_11.score(X_test_11, y_test_11))

In [ ]:
plt.rcParams["figure.figsize"] = (20, 7)
plot_feature_importances(rfc_11, feature_names=x_class_scaled_11.columns, x_tick_rotation=75)

RFC performed reasonably on Category 11 as well, although the bins are less balanced (a lot more zeroes in Category 11). Accuracy scores again shifted a bit when I reran the code with a fresh train/test set.

We discussed potential issues with including both the 2020 and 2021 data in the model because some companies have data for both years (essentially, should be paired samples) while other do not. So I decided to try re-running the models on the 2020 and 2021 data separately to see how they performed.

In [ ]:
#rerunning model on 2020 and 2021 data separately
x_class_scaled_20 = x_class_scaled[x_class_scaled['Data_Year'] < 1]
x_class_scaled_20.info()

In [ ]:
x_bin['Data_Year'].value_counts()

In [ ]:
y_class_20 = y_class[0:328]
y_class_20.astype('category').cat.codes

In [ ]:
X_train_20, X_test_20, y_train_20, y_test_20 = train_test_split(x_class_scaled_20, y_class_20, test_size=0.2)

rfc_20 = RandomForestClassifier(max_depth=4, random_state=42)
rfc_20.fit(X_train_20, y_train_20)

print(rfc_20.score(X_train_20, y_train_20))
print(rfc_20.score(X_test_20, y_test_20))

In [ ]:
y_class_21 = y_class[328::]
y_class_21.astype('category').cat.codes

In [ ]:
x_class_scaled_21 = x_class_scaled[x_class_scaled['Data_Year'] > 1]
x_class_scaled_21.info()

In [ ]:
X_train_21, X_test_21, y_train_21, y_test_21 = train_test_split(x_class_scaled_21, y_class_21, test_size=0.2)

rfc_21 = RandomForestClassifier(max_depth=2, random_state=42)
rfc_21.fit(X_train_21, y_train_21)

print(rfc_21.score(X_train_21, y_train_21))
print(rfc_21.score(X_test_21, y_test_21))

The model worked reasonably well on 2021 data (.76 on training, .75 on test), but had overfitting issues on the 2020 data alone. Additional data from previous years, if available, could help indicate whether 2020 is an anomaly year (very possible with COVID impacts and supply chain issues).
